In [1]:
import os
import numpy as np
import pandas as pd

from skimage.io import imread, imread_collection
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim

## Read all images (64x64)
- Real
- DCGAN
- DCGAN (Style)
- WGAN
- WGAN (Style)
- WGAN-GP
- WGAN-GP (Style)
- UNetGAN-Norm
- UNetGAN-Norm (Style)
- UNetGAN-Ortho
- UNetGAN-Ortho (Style)
- AggrGAN-All
- AggrGAN-All (Style)
- AggrGAN-Top3
- AggrGAN-Top3 (Style)

In [2]:
BASE_DIR = os.getcwd()+"/images"

In [3]:
cls = ["t1", "t1ce"]
models = {
    "DCGAN": "dcgan",
    "DCGAN+style transfer": "dcgan_style",
    "WGAN": "wgan",
    "WGAN+style transfer": "wgan_style",
    "WGAN-GP": "wgan_gp",
    "WGAN-GP+style transfer": "wgan_gp_style",
    "UNetGAN (normal weight init)": "unetgan-norm",
    "UNetGAN (normal weight init)+style transfer": "unetgan_style-norm",
    "UNetGAN (ortho weight init)": "unetgan-ortho",
    "UNetGAN (ortho weight init)+style transfer": "unetgan_style-ortho",
    "AGGrGAN (total agg)": "sobel_aggr_all",
    "AGGrGAN (total agg)+style transfer": "sobel_aggr_all_style",
    "AGGrGAN (top 3 PSNR)": "sobel_aggr_top3",
    "AGGrGAN (top 3 PSNR)+style transfer": "sobel_aggr_top3_style"
}
metrics = ["PSNR", "SSIM"]

output = dict()
for model in models:
    output[model] = dict()
    for cl in cls:
        for metric in metrics:
            output[model][cl.upper()+"_"+metric] = list()

file = dict.fromkeys(cls, dict.fromkeys(models, None))
for cl in cls:
    for model in models:
        model_frags = models[model].split('-')
        if len(model_frags) > 1:
            files = imread_collection(BASE_DIR+f"/{cl}/{model_frags[0]}/{model_frags[1]}/*.png")
        else:
            files = imread_collection(BASE_DIR+f"/{cl}/{models[model]}/*.png")
        for i, f in enumerate(files.files):
            rdir = BASE_DIR+f"/{cl}/real/{i}.png"
            real = imread(rdir, as_gray=True)
            fake = imread(f, as_gray=True)
            # print(f"{rdir} | {f} ==> {real.max()} | {fake.shape}")
            output[model][cl.upper()+"_"+"PSNR"].append(psnr(real, fake, data_range=1.0))
            output[model][cl.upper()+"_"+"SSIM"].append(ssim(real, fake, data_range=1.0))
        output[model][cl.upper()+"_"+"PSNR"] = np.round(np.mean(output[model][cl.upper()+"_"+"PSNR"]),decimals=2)
        output[model][cl.upper()+"_"+"SSIM"] = np.round(np.mean(output[model][cl.upper()+"_"+"SSIM"]),decimals=2)

In [4]:
df = pd.DataFrame.from_dict(output, orient='index')
df = df.rename_axis('Method').reset_index()

In [5]:
df.to_csv('score_table.csv')